# <center>**REPOSITORIO PUCP**</center>



### **Descripción**

El objetivo es obtener información estructurada de las tesis de pregrado y posgrado de Estadística, Informática o de carreras similares desarrolladas a nivel nacional, a través de los repositorios institucionales de las universidades.


El repositorio escogido es el de la Pontificia Universidad Católica del Perú (PUCP).

El respectivo url de dicho repositorio es: https://repositorio.pucp.edu.pe/index/handle/123456789/9139/recent-submissions (Ing. Informática)

## 1. Proceso Web Scrapping

El proceso está dividido en 4 partes:

- Primero, se procede a extraer las rutas de acceso para toda las páginas que contiene las tesis. 
- Segundo, dentro de cada página, se extrae las rutas de acceso para toda las tesis.
- Tercero, para cada tesis, se extrae las rutas de acceso a toda la información.
- Cuarto, de toda la información extraída de cada tesis, se extrae la información relevante para el presente trabajo.

### 1.1 Librerías

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

### 1.2 Extracción de rutas de acceso de las páginas

El URL del repositorio está dividido en páginas. Cada página contiene una lista de 20 tesis. El repositorio contiene un total de 379 tesis. Entonces, se debe acceder a cada página para acceder a cada tesis. 

Para realizar dicha tarea, se presenta el siguiente código:

1. Parsear la `URL` del repositorio PUCP y obtener el esquema (protocolo) y el dominio principal de la `URL`.

2. Realizar una solicitud `HTTP` a la `URL` y obtener el contenido `HTML` de la página.

3. Procesar el contenido `HTML` con la librería `BeautifulSoup`.

4. Buscar el enlace de la siguiente página de resultados en la página actual utilizando el atributo `class` de la etiqueta `a`.

5. Construir una nueva `URL` con el esquema y el dominio principal obtenidos previamente, y agregar el enlace de la siguiente página a la lista de `URL` a procesar.

6. Repetir los pasos 3 a 5 para cada una de las `URL` en la lista hasta que no se encuentre el enlace de la siguiente página.

7. Finalmente, eliminar el último elemento de la lista (que aparece en blanco).


El código recorre y procesa todas las páginas de resultados de una búsqueda en el repositorio PUCP de tesis de ingeniería informática. La lista `lista` almacenaría todos estos enlaces.

In [6]:
# URL del repositorio PUCP: tesis de Ingeniería Informática
url = 'https://repositorio.pucp.edu.pe/index/handle/123456789/9139/recent-submissions'
parsed_url = urlparse(url)

# Hacer solicitud HTTP a la URL y obtener el contenido HTML
response = requests.get(url)
html = response.content

# Procesar el HTML con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Buscamos 
ps = soup.find('a', class_ = 'next-page-link').get('href')

# Obtiene el dominio principal
domain = parsed_url.netloc

# Obtiene el esquema
scheme = parsed_url.scheme

# Construye el URL del dominio principal
url2 = f'{scheme}://{domain}'

lista = []
lista.append(url)
url3 = None
i = 0
while lista[-1] != url2:
    
    url3 = lista[i]

    # Solicitud
    r = requests.get(url3)
    h = r.content
    
    # Procesar
    s = BeautifulSoup(h, 'html.parser')
    
    # Buscamos
    p = s.find('a', class_ = 'next-page-link').get('href')
    
    # Agregamos a la lista
    lista.append(url2 + p)
    i += 1

# Se elimina el último elemento de la lista, que aparece en blanco
lista.pop(-1)

'https://repositorio.pucp.edu.pe'

### 1.3 Extracción de rutas de acceso de las tesis

Ahora que se tiene acceso a cada página del repositorio, necesitamos extraer las rutas de acceso a cada tesis. Para ello se procede de la siguiente forma:

1. Declarar una lista vacía llamada `handle`.
2. Recorrer cada `URL` en la lista de nombre `lista`.
3. Para cada URL:
    1. Realizar una solicitud `HTTP` a la `URL` y obtener el contenido `HTML` de la página.
    2. Procesar el contenido `HTML` con la librería `BeautifulSoup`.
    3. Buscar todas las etiquetas `h4` en el `HTML` procesado.
    4. Recorrer cada etiqueta `h4` y extraer el enlace contenido en la etiqueta `a` que se encuentra dentro de ella.
    5. Agregar el enlace a la lista `handle`.

El código recorre todas las páginas de resultados de una búsqueda en el repositorio PUCP de tesis de ingeniería informática y extrae todos los enlaces a las tesis encontradas en cada página. La lista `handle` almacenaría todos estos enlaces.

In [8]:
handle = []

for i in lista:
    # URL del repositorio PUCP: tesis de Ingeniería Informática
    url4 = i

    # Hacer solicitud HTTP a la URL y obtener el contenido HTML
    response2 = requests.get(url4)
    html2 = response2.content

    # Procesar el HTML con BeautifulSoup
    soup2 = BeautifulSoup(html2, 'html.parser')

    # Extraer todos los enlaces con la etiqueta <h4>
    h4s = soup2.find_all('h4')

    # Iterar sobre cada etiqueta <h4>
    for h4 in h4s:
        # Extraer el enlace de la etiqueta <a> dentro de la etiqueta <h4>
        link = h4.find('a').get('href')
        handle.append(link)

### 1.4 Extracción de rutas de acceso a la información de cada tesis

Ahora que se tiene acceso a cada tesis, necesitamos extraer toda la información disponible de la tesis. Al acceder al enlace de una tesis, se muestra una opción de nombre `Metadatos`, que contiene toda la información requerida. Entonces, necesitamos acceder a esta opción para extraer dicho información. Para ello se procede de la siguiente manera: 

1. Declarar la lista vacía llamada `rutas`.
2. Recorrer cada enlace en la lista `handle`.
3. Para cada enlace:
    1. Realizar una solicitud `HTTP` a la `URL` y obtiene el contenido `HTML` de la página.
    2. Procesar el contenido `HTML` con la librería `BeautifulSoup`.
    3. Buscar todos los enlaces que tengan el texto `"Show full item record"`.
    4. Si se encontraron enlaces, obtener el valor del atributo `href` del primer enlace y agregar a la lista `rutas`.
    
El código recorre cada una de las páginas de las tesis encontradas en el repositorio PUCP y extraer el enlace a la página con la información completa de cada tesis. La lista `rutas` almacenará todos estos enlaces.

In [10]:
rutas = []

for j in handle:
    
    # Obtén el contenido HTML de la página web
    url5 = url2 + j
    page3 = requests.get(url5)
    soup3 = BeautifulSoup(page3.content, 'html.parser')
    # Buscar todos los enlaces que tengan el texto "Show full item record"
    enlaces = soup3.find_all('a', text='Show full item record')

    # Si se encontraron enlaces, obtener el valor del atributo href del primer enlace
    if enlaces:
        href = enlaces[0]['href']
        rutas.append(href)  # imprime el valor del atributo href

### 1.5 Extracción de la información de cada tesis

Finalmente se extrae la información contenida en la opción `Metadatos`, y se filtra la información relevante para el trabajo. Para ello se procede de la siguiente forma:

1. Declarar siete listas vacías llamadas `universidad`, `titulo`, `autor`, `grado`, `asesor`, `resumen` y `anio`.
2. Declarar una lista llamada `v` que contiene etiquetas o claves para algunos campos o atributos que se buscan en las páginas de las tesis.
3. Recorrer cada enlace en la lista `rutas`.
4. Para cada enlace:
    1. Realizar una solicitud `HTTP` a la `URL` y obtener el contenido `HTML` de la página.
    2. Procesar el contenido HTML con la librería BeautifulSoup.
    3. Buscar todas las celdas `td` con la clase `label-cell` y `word-break`.
    4. Almacenar los contenidos de las celdas con la clase `label-cell` en una lista `l` y los contenidos de las celdas con la clase `word-break` en una lista `m`.
    5. Crear un diccionario `diccionario` utilizando las listas `l` y `m` como claves y valores respectivamente.
    6. Revisar si cada una de las etiquetas en la lista `v` están en el diccionario y, de ser así, agregar el valor correspondiente a la lista correspondiente. Si la etiqueta no se encuentra en el diccionario, se agrega el valor `"Sin registro"` a la lista correspondiente.
    

El código recorre cada una de las páginas de las tesis encontradas en el repositorio PUCP y extraer información específica de cada tesis. Las siete listas almacenarán esta información para cada tesis.

In [12]:
universidad = []
titulo = []
autor = []
grado = []
asesor = []
resumen = []
anio = []

v = ['dc.publisher','dc.title','dc.contributor.author','thesis.degree.name','dc.contributor.advisor',
    'dc.description.abstract','dc.date.issued']

for o in rutas:

    q = url2 + o
    p = requests.get(q)
    ss = BeautifulSoup(p.content, 'html.parser')
    ls = ss.find_all('td', class_= 'label-cell')
    tds = ss.find_all('td', class_= 'word-break')

    m = []
    for td in tds:
        c = td.text
        m.append(c)

    l = []
    for g in ls:
        u = g.text
        l.append(u)
    
    diccionario = dict(zip(l,m))
  
    if v[0] in diccionario:
        universidad.append(diccionario[v[0]])
    else:
        universidad.append('Sin registro')

    if v[1] in diccionario:
        titulo.append(diccionario[v[1]])
    else:
        titulo.append('Sin registro')

    if v[2] in diccionario:
        autor.append(diccionario[v[2]])
    else:
        autor.append('Sin registro')

    if v[3] in diccionario:
        grado.append(diccionario[v[3]])
    else:
        grado.append('Sin registro')

    if v[4] in diccionario:
        asesor.append(diccionario[v[4]])
    else:
        asesor.append('Sin registro')

    if v[5] in diccionario:
        resumen.append(diccionario[v[5]])
    else:
        resumen.append('Sin registro')

    if v[6] in diccionario:
        anio.append(diccionario[v[6]])
    else:
        anio.append('Sin registro')

### 1.6 Almacenamiento de la información

Finalmente, almacenamos la información de forma estructurada. Procedemos de la siguiente forma:

1. Crear un diccionario llamado `datos` donde cada clave es el nombre de un campo o atributo y cada valor es una lista con los valores para ese campo o atributo.
2. Crear un DataFrame de `pandas` llamado `df` a partir del diccionario `datos`.
3. Exportar el DataFrame a un archivo `CSV` llamado `"tesis.csv"` utilizando como separador de campo el caracter `;` y como codificación de caracteres `utf-8`.


El código crea un archivo `CSV` con la información de las tesis extraídas del repositorio PUCP. Cada fila del archivo corresponde a una tesis y cada columna corresponde a un campo o atributo de la tesis.

In [13]:
datos = {
    'Universidad': universidad,
    'Titulo': titulo,
    'Autor': autor,
    'Grado': grado,
    'Asesor': asesor,
    'Resumen': resumen,
    'Año': anio
}

df = pd.DataFrame(datos)
df.to_csv('pucp.csv', sep = ';', index = False, encoding = 'utf-8')

## 2. Resultados

In [16]:
df = pd.read_csv('pucp.csv', sep = ';')
df

,Universidad,Titulo,Autor,Grado,Asesor,Resumen,Año
0,Pontificia Universidad Católica del Perú,Implementación de un sistema de información pa...,"Canasa Mayta, Paul Rodrigo",Ingeniero Informático,"Tupia Anticona, Manuel Francisco",Un almacén cuenta con tres funciones important...,2022-10-13
1,Pontificia Universidad Católica del Perú,Implementación de sistemas de gestión de segur...,"Cabello Roca, Percy Ernesto",Ingeniero Informático,"Flores García, Luis Alberto",El presente informe describe la experiencia ad...,2022-10-13
2,Pontificia Universidad Católica del Perú,Automatización del proceso de gestión de órden...,"Alpiste Sarmiento, Daniel Mijail",Ingeniero Informático,"Murillo Veliz, Braulio Oscar",Desde el inicio del siglo XXI y con el desarro...,2022-09-05
3,Pontificia Universidad Católica del Perú,Implementación de un Sistema de Información Ho...,"Arrué Pajares, Sebastián Daniel",Ingeniero Informático,"Lena Valega, Ángel Gabriel","Dada la coyuntura actual del COVID-19, en el P...",2022-08-17
4,Pontificia Universidad Católica del Perú,Diseño de un modelo para identificar amenazas ...,"Castillo Lopez, Anderson Jesús",Ingeniero Informático,"Tupia Anticona, Manuel Francisco","En la actualidad, el uso de las tecnologías de...",2022-08-11
...,...,...,...,...,...,...,...
338,Pontificia Universidad Católica del Perú,"Análisis, diseño y construcción de una herrami...","Rondón Suasnabar, Sandro Salvador",Ingeniero Informático,Sin registro,En este trabajo de tesis se presenta el desarr...,2011-05-09
339,Pontificia Universidad Católica del Perú,"Análisis, diseño e implementación de un sistem...","Chanca de la Cruz, Marllessi Marilyn",Ingeniero Informático,Sin registro,El presente proyecto de tesis tiene por objeti...,2011-05-09
340,Pontificia Universidad Católica del Perú,Modelado lingüístico-prosódico para un sistema...,"Segura Salas, Elí Rónal",Ingeniero Informático,Sin registro,El objeto principal de esta tesis ha sido el e...,2011-05-09
341,Pontificia Universidad Católica del Perú,Plataforma Web de apoyo a las PYMES para la ge...,"Rosas Limaylla, Jhénifer Florinda",Ingeniero Informático,Sin registro,El sector de las PYMES es el motor de la econo...,2011-05-09


## 3. Algunos Estadísticos

### 3.1 Cantidad de tesis sin asesor

In [24]:
df2 = df[df['Asesor'] == 'Sin registro']
print(len(df2))

92


### 3.2 Cantidad de tesis por cada asesor

In [27]:
df3 = df.groupby('Asesor')['Asesor'].count()

df3_ordenado = df3.sort_values(ascending=False)

print(df3_ordenado)

Asesor
Sin registro                               92
Cueva Moscoso, Rony                        34
Melgar Sasieta, Héctor Andrés              22
Beltrán Castañón, César Armando            22
Flores García, Luis Alberto                13
                                           ..
Murillo Veliz, Braulio                      1
Olivares Poggi, César Augusto               1
Davila, Abraham                             1
Palma Stanciuc, Rossana Inés                1
Zapata Del Río, Claudia María del Pilar     1
Name: Asesor, Length: 66, dtype: int64
